In [2]:
import pandas as pd
import cv2 as cv
import numpy as np
from scipy.stats import skew

In [3]:
train_df = pd.read_csv("./dataset/splits/train.csv")
train_df.head()

,Unnamed: 0,clip_name,clip_path,label,encoded_label
0,0,v_Diving_g07_c04.avi,./dataset/Diving/v_Diving_g07_c04.avi,Diving,0
1,1,v_Diving_g05_c02.avi,./dataset/Diving/v_Diving_g05_c02.avi,Diving,0
2,2,v_Diving_g22_c05.avi,./dataset/Diving/v_Diving_g22_c05.avi,Diving,0
3,3,v_Diving_g12_c02.avi,./dataset/Diving/v_Diving_g12_c02.avi,Diving,0
4,4,v_Diving_g07_c01.avi,./dataset/Diving/v_Diving_g07_c01.avi,Diving,0


In [4]:
def manual_normalize(frame):
    mean, std = cv.meanStdDev(frame)
    mean = mean.flatten()
    std = std.flatten()
    return (resized - mean) / std

def open_cv_normalize(frame):
    return cv.normalize(frame, None, alpha=0, beta=1.0, norm_type=cv.NORM_MINMAX, dtype=cv.CV_32F)

def open_cv_normalization_255(frame):
    return cv.normalize(frame, None, alpha=0, beta=255,
                               norm_type=cv.NORM_MINMAX, dtype=cv.CV_8U)


In [5]:
def color_segmentation(frame):
    rgb_frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    hsv_frame = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
    resized_rgb = cv.resize(rgb_frame, (224, 224), interpolation=cv.INTER_CUBIC)
    resized_hsv = cv.resize(hsv_frame, (224, 224), interpolation=cv.INTER_CUBIC)

    hist_features = {}
    for i, col in enumerate(['red', 'green', 'blue']):
        hist = cv.calcHist(resized_rgb, [i], None, [256], [0, 256])
        hist_features[col] = hist.flatten()

    moments = []
    for i in range(3):
        channel = resized_hsv[:, :, i]
        mean = np.mean(channel)
        std = np.std(channel)
        skewness = skew(channel)
        moments.append([mean, std, skewness])

    avg_color = np.mean(resized_rgb, axis=(0, 1))
    return hist_features, moments, avg_color



In [9]:
color_df = pd.DataFrame(columns=['index', 'moments_mean_h', 'moments_std_h', 'moments_skew_h', 'moments_mean_s', 'moments_std_s', 'moments_skew_s', 'moments_mean_v', 'moments_std_v', 'moments_skew_v', 'avg_color_r', 'avg_color_g', 'avg_color_b'])

def add_color_feature_pipeline(index, file_path):
    file = cv.VideoCapture(file_path)
    avg_moments = [0, 0, 0, 0, 0, 0, 0, 0, 0]
    avg_colors = [0, 0, 0]
    while file.isOpened():
        if not file:
            print("Error opening video stream or file")
        ret, frame = file.read()

        if ret:
            hist_features, moments, avg_color = color_segmentation(frame)
            avg_moments[0] += moments[0][0]
            avg_moments[1] += moments[0][1]
            avg_moments[2] += moments[0][2]
            avg_moments[3] += moments[1][0]
            avg_moments[4] += moments[1][1]
            avg_moments[5] += moments[1][2]
            avg_moments[6] += moments[2][0]
            avg_moments[7] += moments[2][1]
            avg_moments[8] += moments[2][2]
            avg_colors[0] += avg_color[0]
            avg_colors[1] += avg_color[1]
            avg_colors[2] += avg_color[2]
        else:
            break
    total_frames = file.get(cv.CAP_PROP_FRAME_COUNT)
    avg_moments[0] /= total_frames
    avg_moments[1] /= total_frames
    avg_moments[2] /= total_frames
    avg_moments[3] /= total_frames
    avg_moments[4] /= total_frames
    avg_moments[5] /= total_frames
    avg_moments[6] /= total_frames
    avg_moments[7] /= total_frames
    avg_moments[8] /= total_frames
    avg_colors[0] /= total_frames
    avg_colors[1] /= total_frames
    avg_colors[2] /= total_frames
    color_df.append({'index': index,
        'moments_mean_h': avg_moments[0], 'moments_std_h': avg_moments[1], 'moments_skew_h': avg_moments[2],
        'moments_mean_s': avg_moments[3], 'moments_std_s': avg_moments[4], 'moments_skew_s': avg_moments[5],
        'moments_mean_v': avg_moments[6], 'moments_std_v': avg_moments[7], 'moments_skew_v': avg_moments[8],
        'avg_color_r': avg_colors[0], 'avg_color_g': avg_colors[1], 'avg_color_b': avg_colors[2]}, ignore_index=True)
    file.release()


In [6]:
file = cv.VideoCapture(train_df.iloc[0].clip_path)



frames_queue = []
while file.isOpened():
    ret, frame = file.read()

    if ret:
        color_segmentation_pipeline(frame)
        color_df.append({'index': train_df.iloc[0].index,
                     'moments_mean_h': moments[0][0], 'moments_std_h': moments[0][1], 'moments_skew_h': moments[0][2],
                     'moments_mean_s': moments[1][0], 'moments_std_s': moments[1][1], 'moments_skew_s': moments[1][2],
                     'moments_mean_v': moments[2][0], 'moments_std_v': moments[2][1], 'moments_skew_v': moments[2][2],
                     'avg_color_r': avg_color[0], 'avg_color_g': avg_color[1], 'avg_color_b': avg_color[2]}, ignore_index=True)

        resized = cv.resize(frame, (224, 224), interpolation=cv.INTER_CUBIC)
        resized = open_cv_normalization_255(resized)

        frames_queue.append(resized)
        if len(frames_queue) >= 5:
            deionised = cv.fastNlMeansDenoisingColoredMulti(
                srcImgs=frames_queue,
                imgToDenoiseIndex=2,
                temporalWindowSize=5,
                h=10,
                hColor=10,
                templateWindowSize=7,
                searchWindowSize=21
            )
            sharpening_kernel = np.array([[-1, -1, -1],
                                  [-1,  9, -1],
                                  [-1, -1, -1]])
            sharp_img = cv.filter2D(deionised, -1, sharpening_kernel)
            cv.imshow('Deionised', sharp_img)
            frames_queue.pop(0)

        print(resized.shape)
        cv.imshow('Frame', frame)
        cv.imshow('Resized', resized)
        # cv.imshow('Resized2', resized2)
        if cv.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break

file.release()
cv.destroyWindow('Frame')
cv.destroyWindow('Resized')
cv.destroyWindow('Deionised')
# cv.destroyWindow('Resized2')


(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)


QFontDatabase: Cannot find font directory /home/yogendra/workspace/python/university/Video-Analytics-Assignment1/.venv/lib/python3.12/site-packages/cv2/qt/fonts.
Note that Qt no longer ships fonts. Deploy some (from https://dejavu-fonts.github.io/ for example) or switch to fontconfig.
QFontDatabase: Cannot find font directory /home/yogendra/workspace/python/university/Video-Analytics-Assignment1/.venv/lib/python3.12/site-packages/cv2/qt/fonts.
Note that Qt no longer ships fonts. Deploy some (from https://dejavu-fonts.github.io/ for example) or switch to fontconfig.
QFontDatabase: Cannot find font directory /home/yogendra/workspace/python/university/Video-Analytics-Assignment1/.venv/lib/python3.12/site-packages/cv2/qt/fonts.
Note that Qt no longer ships fonts. Deploy some (from https://dejavu-fonts.github.io/ for example) or switch to fontconfig.
QFontDatabase: Cannot find font directory /home/yogendra/workspace/python/university/Video-Analytics-Assignment1/.venv/lib/python3.12/site-pac

(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 

In [8]:
print(moments)

[[np.float64(88.11334103954081), np.float64(55.57221405305902), array([ 1.02436996,  1.02436996,  1.02436996,  1.01813555,  1.02373463,
        1.03229191,  1.02586703,  1.02269568,  1.00730164,  1.01954868,
        1.03733404,  0.73889813,  0.70938716,  0.76619811,  0.86448655,
        0.85137564,  0.83465652,  0.83324675,  0.83336683,  0.83336819,
        0.83342048,  0.84622416,  0.72634828,  0.69610155,  0.69709088,
        0.59958609,  0.57938452,  0.53586225,  0.52725344,  0.5482541 ,
        0.5139996 ,  0.5377543 ,  0.56497172,  0.55744361,  0.50472569,
        0.49836077,  0.49941849,  0.49981783,  0.49993003,  0.49894793,
        0.49894793,  0.49867678,  0.50020279,  0.50574154,  0.49214183,
       -0.20342837, -0.20287376, -0.20238189, -0.20211917, -0.20236895,
       -0.20199039, -0.20183593, -0.20045599, -0.23887648, -0.27538271,
       -0.22905152, -0.14678182, -0.13881651, -0.13882004, -0.13652967,
       -0.13798238, -0.1379493 , -0.13803194, -0.1267685 ,  0.05387977,
